# Characterizing and Predicting Dog Adoption Rates

I am using data sets I obtained from Kaggle: [dog breeds parameters scraped from at dogtime.com](https://www.kaggle.com/hocop1/cat-and-dog-breeds-parameters) and the Austin Animal Center's [intake and outake records.](https://www.kaggle.com/aaronschlegel/austin-animal-center-shelter-intakes-and-outcomes/) 

In [38]:
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from zipfile import ZipFile

In [4]:
!cd ../data_sets/ & ls

austin-animal-center-shelter-intakes-and-outcomes.zip
BikeShare
Bikeshare-Data.zip
cat-and-dog-breeds-parameters.zip
chicago-red-light-and-speed-camera-data.zip
example_files
Fred_CPI_Foodathome_CUSR0000SAF11.csv
music_caps
NYC
ny-city-owned-and-leased-property.zip
OnTheSnow_v1.xlsx
ramen-ratings.csv
ramen-ratings.zip
Readme_file.docx
sentiment_sentences
sentiment_sentences.zip
speed_cam
WELLCOME_APCspend2013_forThinkful.csv


In [5]:
with ZipFile(r'../data_sets/austin-animal-center-shelter-intakes-and-outcomes.zip', 'r') as zip:
    zip.extractall()
with ZipFile(r'../data_sets/cat-and-dog-breeds-parameters.zip', 'r') as zip:
    zip.extractall()

In [7]:
ls

 Volume in drive C is Windows
 Volume Serial Number is 6839-6E71

 Directory of C:\Users\Chris\Documents\thinkful\supervised_learning_capstone

08/03/2019  02:44 PM    <DIR>          .
08/03/2019  02:44 PM    <DIR>          ..
08/03/2019  02:04 PM    <DIR>          .ipynb_checkpoints
08/03/2019  02:42 PM        13,040,169 aac_intakes.csv
08/03/2019  02:42 PM        31,441,583 aac_intakes_outcomes.csv
08/03/2019  02:42 PM        11,648,624 aac_outcomes.csv
08/03/2019  02:04 PM             2,357 Capstone_supervised_learning_notes.ipynb
08/03/2019  02:44 PM             3,935 dog_adoption_SL_capstone.ipynb
08/03/2019  02:42 PM           271,269 rating.json
               6 File(s)     56,407,937 bytes
               3 Dir(s)  28,655,845,376 bytes free


In [54]:
shelter_inout_df = pd.read_csv(r'aac_intakes_outcomes.csv')  # I will used the merged file with both intake records and outcome records
shelter_inout_df.head()

,age_upon_outcome,animal_id_outcome,date_of_birth,outcome_subtype,outcome_type,sex_upon_outcome,age_upon_outcome_(days),age_upon_outcome_(years),age_upon_outcome_age_group,outcome_datetime,...,age_upon_intake_age_group,intake_datetime,intake_month,intake_year,intake_monthyear,intake_weekday,intake_hour,intake_number,time_in_shelter,time_in_shelter_days
0,10 years,A006100,2007-07-09 00:00:00,NaN,Return to Owner,Neutered Male,3650,10.0,"(7.5, 10.0]",2017-12-07 14:07:00,...,"(7.5, 10.0]",2017-12-07 00:00:00,12,2017,2017-12,Thursday,14,1.0,0 days 14:07:00.000000000,0.588194
1,7 years,A006100,2007-07-09 00:00:00,NaN,Return to Owner,Neutered Male,2555,7.0,"(5.0, 7.5]",2014-12-20 16:35:00,...,"(5.0, 7.5]",2014-12-19 10:21:00,12,2014,2014-12,Friday,10,2.0,1 days 06:14:00.000000000,1.259722
2,6 years,A006100,2007-07-09 00:00:00,NaN,Return to Owner,Neutered Male,2190,6.0,"(5.0, 7.5]",2014-03-08 17:10:00,...,"(5.0, 7.5]",2014-03-07 14:26:00,3,2014,2014-03,Friday,14,3.0,1 days 02:44:00.000000000,1.113889
3,10 years,A047759,2004-04-02 00:00:00,Partner,Transfer,Neutered Male,3650,10.0,"(7.5, 10.0]",2014-04-07 15:12:00,...,"(7.5, 10.0]",2014-04-02 15:55:00,4,2014,2014-04,Wednesday,15,1.0,4 days 23:17:00.000000000,4.970139
4,16 years,A134067,1997-10-16 00:00:00,NaN,Return to Owner,Neutered Male,5840,16.0,"(15.0, 17.5]",2013-11-16 11:54:00,...,"(15.0, 17.5]",2013-11-16 09:02:00,11,2013,2013-11,Saturday,9,1.0,0 days 02:52:00.000000000,0.119444


In [53]:
with open('rating.json', 'r') as rating_file:
    ratings = json.load(rating_file)
    json_normalize(ratings)
    dog_descr_df = pd.DataFrame.from_dict(ratings['dog_breeds'], ).T
dog_descr_df.head()

,Adaptability,All Around Friendliness,Exercise Needs,Health Grooming,Trainability,Adapts Well to Apartment Living,Affectionate with Family,Amount Of Shedding,Dog Friendly,Drooling Potential,...,Potential For Playfulness,Potential For Weight Gain,Prey Drive,Sensitivity Level,Size,Tendency To Bark Or Howl,Tolerates Being Alone,Tolerates Cold Weather,Tolerates Hot Weather,Wanderlust Potential
Affenpinscher,3.0,3.0,4.0,2.0,3.0,5.0,5.0,1.0,4.0,1.0,...,4.0,3.0,3.0,3.0,1.0,2.0,1.0,3.0,3.0,2.0
Afghan Hound,4.0,4.0,4.0,2.0,3.0,5.0,5.0,4.0,4.0,1.0,...,4.0,1.0,5.0,5.0,4.0,2.0,2.0,5.0,5.0,5.0
Airedale Terrier,2.0,4.0,5.0,3.0,5.0,1.0,4.0,2.0,4.0,1.0,...,5.0,4.0,5.0,3.0,3.0,4.0,2.0,3.0,3.0,4.0
Akita,3.0,2.0,4.0,4.0,4.0,2.0,5.0,5.0,1.0,5.0,...,5.0,4.0,4.0,5.0,4.0,5.0,1.0,5.0,2.0,4.0
Alaskan Klee Kai,3.0,3.0,4.0,3.0,4.0,3.0,4.0,4.0,2.0,2.0,...,4.0,2.0,5.0,4.0,2.0,3.0,2.0,5.0,2.0,4.0
